<a href="https://colab.research.google.com/github/adhalya/Curso_PySpark/blob/main/PySpark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# Resolver problema de versão python x pyspark worker

In [6]:
from pyspark.sql import SparkSession # Iniciar sessão do PySpark
from pyspark.sql.functions import * # Importa algumas funções
from pyspark.sql.types import * # Funções que transformam colunas em tipos específicos

**Criar/Iniciar Sessão PySpark**

In [7]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

**Criar df/ Ler arquivo**

In [8]:
df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True) # Cria cabeçalho e define o tipo de cada coluna, se não tiver vai considerar todas como string

**Exibir dataframe**

In [9]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



**Verificar tipo de colunas**

In [10]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



**Verificando dados nulos**

In [11]:
df.toPandas().isna().sum() #toPandas não funciona bem para df's maiores

,0
Team,0
#,0
Pos.,0
FIFA Popular Name,0
Birth Date,0
Shirt Name,0
Club,0
Height,0
Weight,0


**Renomeando colunas**

In [12]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time')\
.withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight', 'Peso')
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [13]:
for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

# Esse for não considera pontos, por isso foi necessário renomear as colunas para ele funcionar
# Como ele vai ser executado linha por linha, o processo é demorado em dataframes grandes

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


**Selecionar Colunas**

In [14]:
df.select('Selecao','Nome_FIFA').show(5)
# Apesar de executar mais rápido, selecionando dessa forma, impossibilita de usar o ALIAS

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [15]:
df.select(col('Selecao'), col('Altura')).show(5)

+---------+------+
|  Selecao|Altura|
+---------+------+
|Argentina|   169|
|Argentina|   169|
|Argentina|   167|
|Argentina|   167|
|Argentina|   170|
+---------+------+
only showing top 5 rows



In [16]:
df.select(df['Selecao']).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



**Selecionar Colunas com ALIAS**

In [17]:
df.select(df['Selecao'].alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [18]:
df.select('Selecao Nome_FIFA Altura'.split()).show(5)
# Não é muito utilizado

+---------+------------------+------+
|  Selecao|         Nome_FIFA|Altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



**Organizar Select**

In [19]:
df.select('Nome_FIFA', 'Peso', 'Altura').show(5)

# O código acima tem a intenção de demonstrar que é possível organizar as colunas de acordo com a ordem selecionada

+------------------+----+------+
|         Nome_FIFA|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



**Filtrar DF**

Se atribuir o filtro ao dataframe, os dados originais irão se perder, o que deve fazer caso não queira essa opção é não realizar a atribuição, apenas visualizar os dados em tela, ou, guardar o filtro em uma nova variável.

In [20]:
df.filter('Selecao == "Brazil"').show(10)

# Com a seleção acima, não é possível filtrar nomes de colunas com espaço

+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|     D. COSTA|   Juventus FC (ITA)|   

In [21]:
df.filter(col('Nome Camiseta') == 'FRED').show()
# Utilizando o filtro dessa forma é possível encontrar colunas com espaços

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



**Filtrar DF com 2 condições (AND / &)**

In [22]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [23]:
df.filter((col('Selecao') == "Argentina") & (col('Altura') > 180) & (col('Peso') >= 85)).show()

+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|Argentina|     6|     DF|FAZIO Federico|17.03.1987|        FAZIO|       AS Roma (ITA)|   199|  85|
|Argentina|    12|     GK| ARMANI Franco|16.10.1986|       ARMANI|CA River Plate (ARG)|   189|  85|
|Argentina|     1|     GK| GUZMAN Nahuel|10.02.1986|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+



In [24]:
df.filter('Selecao == "Brazil"').filter(col('Numero') > 20).show(5)

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    21|     FW|   TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    22|     DF|   FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    23|     GK|  EDERSON|17.08.1993|      EDERSON|Manchester City F...|   188|  86|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



**Filtrar DF com 2 condições (OR/|)**

In [25]:
df.filter((col('Nome_FIFA') == 'MESSI Lionel') | (col('Nome_FIFA') == 'SALVIO Eduardo') | (col('Altura') == 199)).show(5)

+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|              Time|Altura|Peso|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|Argentina|    18|     DF|  SALVIO Eduardo|13.07.1990|       SALVIO|  SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|    MESSI Lionel|24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
|Argentina|     6|     DF|  FAZIO Federico|17.03.1987|        FAZIO|     AS Roma (ITA)|   199|  85|
|  Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|  Chelsea FC (ENG)|   199|  91|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+



**Filtrar DF combinando And e OR (& / |)**

In [26]:
df.filter((col('Selecao') == 'Brazil') & (col('Posicao') == 'DF') | (col('Altura') == 199) & (col('Selecao') == 'Belgium')).show()

+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|    Chelsea FC (ENG)|   199|  91|
| Brazil|    22|     DF|          FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|     6|     DF|     FILIPE LUIS|09.08.1985|  FILIPE LUIS|Atletico Madrid (...|   182|  73|
| Brazil|    13|     DF|      MARQUINHOS|14.05.1994|   MARQUINHOS|Paris Saint-Germa...|   183|  75|
| Brazil|     3|     DF|         MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
| Brazil|    14|     DF|          DANILO|15.07.1991|       DANILO|Manchester City F...|   184|  78|
| Brazil|     2|     DF|    THIAGO SILVA|22.09.1984|     T. SILVA|Paris Saint-Germa...|   183|  79|


**Criar novas colunas (usando função lit)**

In [27]:
df.withColumn('Coluna_nova', lit(col('Altura') - col('Peso'))).show(5)
# O lit (literalmente) vai agregar algum valor a uma nova coluna, sendo possível realizar cálculos

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Coluna_nova|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|        104|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|        104|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|        101|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         98|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|         98|
+---------+------+-------+------------------+----------+-------------+--

**Criar coluna condicional (usando função substring)**

In [28]:
df.withColumn('Sub', substring(col('Selecao'), 1, 3)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Sub|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
only showing top 5 rows



In [29]:
df.show(10)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     5|     MF|      BIGLIA Lucas|30.01.1986|       B

In [30]:
df = df.withColumn('Ano', substring('Nascimento', -4,4))

In [31]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

**Criar coluna condicional (concat / concat_ws)**

In [32]:
df.withColumn('Concat', concat('Selecao', 'Nome_FIFA')).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|              Concat|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|ArgentinaTAGLIAFI...|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|ArgentinaPAVON Cr...|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|ArgentinaLANZINI ...|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|ArgentinaSALVIO E...|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)

In [33]:
df.withColumn('Separador', concat_ws(' - ', 'Selecao', 'Numero', 'Posicao')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|          Separador|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| Argentina - 3 - DF|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|Argentina - 22 - MF|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|Argentina - 15 - MF|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|Argentina - 18 - DF|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170

**Alterar tipo de coluna**

In [34]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

In [35]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



In [36]:
df = df.withColumn('Ano', col('Ano').cast(IntegerType()))

In [37]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



**Desafio: Coluna Nascimento**
- Colocar como DateType
- Dica:
- O formato precias estar em YYYY.MM.DD

In [38]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

In [39]:
dia = udf(lambda data: data.split('.')[0])

In [40]:
df = df.withColumn('Dia', dia('Nascimento'))

In [41]:
mes = udf(lambda data: data.split('.')[1])

df = df.withColumn('Mês', mes('Nascimento'))

In [42]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|Dia|Mês|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| 31| 08|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996| 21| 01|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993| 15| 02|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990| 13| 07|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987| 24| 06|
+---------+------+-------+------------------+----------+--------

In [43]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mês: string (nullable = true)



In [44]:
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mês', 'Dia'))

In [45]:
df = df.withColumn('Data_Nascimento', col('Data_Nascimento').cast('date'))

In [46]:
#df.withColumn('Data_Nascimento', to_date('Data_Nascimento', 'yyyy-MM-dd')).show(5)

In [47]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mês: string (nullable = true)
 |-- Data_Nascimento: date (nullable = true)

